In [65]:
# Imports for scraping RT
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from collections import defaultdict


import re

# from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import time
import requests


# Imports for data formatting
import uuid
import hashlib
import json
from datetime import datetime, date


In [66]:
def get_soup(url):
    try:
        time.sleep(1)
        response = requests.get(url)
        if response.url[-5:] == url[-5:]:
            soup = BeautifulSoup(response.text, 'html.parser')
            return soup
    except Exception as e:
        print("url  " + url + " raised exception " + str(e))
        return ""


soup = get_soup("https://battleroyalewithcheese.com/2020/12/the-dry-the-brwc-review/")


In [67]:
def scrape_soup(soup):
    print(soup)
    """
    Scrapes and process text from a webpage, focusing on <p> tags.

    Parameters:
    url (str): The URL of the webpage to scrape.

    Returns:
    str: The cleaned-up text extracted from the web page, or an empty string if an issue occurs during scraping.
    """
    
    # returns class attributes of an HTML element or "NOCLASS" if it doesn't have any.
    def get_class(p):
        return ''.join(p['class']) if p.has_attr('class') else "&&NOCLASS&&"
    
    p_dict = defaultdict(list)
# try:
    for element in soup.find_all():
        if element.attrs and 'class' in element.attrs:
            if any("caption" in class_name.lower() 
                    for class_name in element['class']):
                element.decompose()
            elif any("ssbp-wrap" in class_name.lower() for class_name in element['class']):
                # print("found share-text")
                for sibling in element.find_parent().find_parent().find_next_siblings():
                    sibling.decompose()
                element.decompose()
                break
        # if element.name in ['ol', 'ul', 'dl']:
        #     element.decompose()

    # finds all <p> tags
    paragraphs = soup.find_all('p')
    for p in paragraphs:
        if len(p.text.split(' ')) > 5 and not p.find(['ol', 'ul', 'dl' ]):
            p_dict[get_class(p)].append(p.text)

    # Find the class with the longest list of paragraphs
    longest_key = max(p_dict, key=lambda k: len(p_dict[k]))

    # Combine and clean text
    full_text = re.sub(r'\s+', ' ', ' '.join(p_dict[longest_key])).strip()

# except Exception as e:
    # print("Exception "  + str(e) + " occured")
    # return ""


    return full_text


In [68]:
# def scrape_soup(soup):
#     """
#     Scrapes and process text from a webpage, focusing on <p> tags.

#     Parameters:
#     url (str): The URL of the webpage to scrape.

#     Returns:
#     str: The cleaned-up text extracted from the web page, or an empty string if an issue occurs during scraping.
#     """
    
#     # returns class attributes of an HTML element or "NOCLASS" if it doesn't have any.
#     def get_class(p):
#         return ''.join(p['class']) if p.has_attr('class') else "&&NOCLASS&&"

#     p_dict = defaultdict(list)
#     try:
#         for element in soup.find_all():
#             if element.has_attr('class') and any("caption" in class_name.lower() or "newsletter" in class_name.lower() 
#                                                  for class_name in element['class']):
#                 element.extract()
#         # finds all <p> tags
#         paragraphs = soup.find_all('p')

#         for p in paragraphs:
#             if len(p.text.split(' ')) > 5:
#                 p_dict[get_class(p)].append(p.text)

#         # Find the class with the longest list of paragraphs
#         longest_key = max(p_dict, key=lambda k: len(p_dict[k]))

#         # Combine and clean text
#         full_text = re.sub(r'\s+', ' ', ' '.join(p_dict[longest_key])).strip()

#     except Exception as e:
#         # print("Exception "  + str(e) + " occurred for url: " + url)
#         return ""


#     return full_text


In [69]:
# print(soup)
print(scrape_soup(soup))


<!DOCTYPE html>

<html lang="en-GB">
<head>
<meta charset="utf-8"/>
<link href="http://gmpg.org/xfn/11" rel="profile"/>
<link href="http://battleroyalewithcheese.com/xmlrpc.php" rel="pingback"/>
<meta content="width=device-width,initial-scale=1,user-scalable=no" name="viewport"/>
<!--||  JM Twitter Cards by jmau111 v12  ||-->
<meta content="summary_large_image" name="twitter:card"/>
<meta content="@brwcheese" name="twitter:creator"/>
<meta content="@brwcheese" name="twitter:site"/>
<meta content="The Dry: The BRWC Review" name="twitter:title"/>
<meta content="Australian cinema is oftentimes its own worst enemy, and it comes down to a multitude of reasons as to why. Primarily Australian cinemas biggest sin is that the largest and most prominent domestic fi" name="twitter:description"/>
<meta content="http://battleroyalewithcheese.com/wp-content/uploads/2020/12/Eric.jpg" name="twitter:image"/>
<!--||  /JM Twitter Cards by jmau111 v12  ||-->
<meta content="index, follow, max-image-preview

In [70]:

# # Read the JSONL file and store JSON objects in a list
# with open('output/rtcorpus.jsonl', 'r') as file:
#     json_list = [json.loads(line) for line in file]

# # Create a set to store unique "fullText" values
# unique_fullText_values = set()

# # Create a new list to store filtered JSON objects
# filtered_json_list = []

# # Iterate through the JSON objects
# for json_obj in json_list:
#     # Extract the "fullText" value
#     fullText_value = json_obj.get("fullText", "")

#     # Check if the "fullText" value is unique
#     if fullText_value not in unique_fullText_values:
#         unique_fullText_values.add(fullText_value)
#         filtered_json_list.append(json_obj)

# # Write the filtered JSON objects back to the JSONL file
# with open('output/newcorpus.jsonl', 'w') as file:
#     for json_obj in filtered_json_list:
#         file.write(json.dumps(json_obj) + '\n')
